# **Модуль Б**. Разработка модели машинного обучения

## Импортирование библиотек

In [1]:
# модули для работы с моделью
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd

from IPython.display import Audio

from Model.tokenizer import Tokenizer

import numpy as np

## Загрузка данных

In [2]:
# путь к обработанным данным
df_path = '../Module1/Dataset/dataset.h5'
# загружаем датасета
df = pd.read_hdf(df_path, key='df')
df.head()

,path,sentence,record_duration,sampling_rate,array,text_length,snr,rms_dB,tokens,token_ids,mel_spec
0,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,"Абай был не только талантливым поэтом, но и уч...",5.04,16000,"[-1.4188117e-10, 4.3655746e-11, -8.0035534e-11...",51,29.652619,-21.792862,"[абай, _, был, _, не, _, только, _, талантливы...","[2106, 1888, 6009, 1888, 5554, 1888, 89, 1888,...","[[-13.040604, -10.495331, -7.2823877, -6.36562..."
1,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Гибкая кошка легко взбирается на высокое дерево.,5.65,16000,"[6.91216e-11, -1.0186341e-10, 1.7462298e-10, -...",48,27.797587,-23.426945,"[гибкая, _, кошка, _, легко, _, взбирается, _,...","[155, 1888, 5422, 1888, 1231, 1888, 1584, 1888...","[[-3.221048, -4.417929, -6.569694, -5.2323184,..."
2,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Сделать нужно гораздо больше.,3.64,16000,"[-7.2759576e-11, -8.0035534e-11, -5.820766e-11...",29,27.732599,-24.547728,"[сделать, _, нужно, _, гораздо, _, больше]","[4582, 1888, 2504, 1888, 284, 1888, 2411, 6674...","[[-3.1968756, -3.3615072, -3.878891, -5.443349..."
3,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,"Эти ноты, сливаясь воедино, образуют симфонию,...",6.62,16000,"[7.2759576e-11, -8.367351e-11, 0.0, -7.2759576...",77,23.927073,-25.052473,"[эти, _, ноты, _, сливаясь, _, воедино, _, обр...","[702, 1888, 2786, 1888, 4284, 1888, 3097, 1888...","[[-12.805177, -11.202822, -9.942761, -6.541317..."
4,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Квадрат гипотенузы равен сумме квадратов катетов.,5.83,16000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",49,35.466933,-17.979994,"[квадрат, _, гипотенузы, _, равен, _, сумме, _...","[3793, 1888, 2552, 1888, 2130, 1888, 4431, 188...","[[-5.9685955, -4.9145446, -4.8844066, -4.73808..."


In [11]:
text_example = df['sentence'][42]
text_example

'Европейский союз будет оставаться наблюдателем в Генеральной Ассамблее.'

**Класс с моделью**

In [115]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TTSTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, n_mels=80, time_steps=429):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        
        # Encoder
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead=4),
            num_layers=3
        )
        
        # Decoder: предсказывает только `n_mels`
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 512),
            nn.ReLU(),
            nn.Linear(512, n_mels)  # (batch, seq_len, n_mels)
        )

        # Исправленный Conv1d: адаптируем `seq_len → time_steps`
        self.time_projection = nn.Conv1d(
            in_channels=n_mels, 
            out_channels=n_mels, 
            kernel_size=3, 
            stride=1, 
            padding=1
        )

        self.n_mels = n_mels
        self.time_steps = time_steps

    def forward(self, tokens):
        # print('tokens', tokens)
        x = self.embedding(tokens)  # (batch, seq_len, d_model)
        # print('x1', x.shape)
        x = self.encoder(x)         # (batch, seq_len, d_model)
        # print('x2', x.shape)
        x = self.decoder(x)         # (batch, seq_len, n_mels)
        # print('x3', x.shape)
        
        # Транспонируем для Conv1D: (batch, seq_len, n_mels) → (batch, n_mels, seq_len)
        x = x.permute(0, 2, 1)

        # Используем Conv1D для приведения seq_len к time_steps
        x = F.interpolate(x, size=self.time_steps, mode="linear", align_corners=True)

        # Возвращаем обратно: (batch, n_mels, time_steps) → (batch, time_steps, n_mels)
        x = x.permute(0, 2, 1)

        return x


In [116]:
class TTSDataset(Dataset):
    def __init__(self, tokenized_texts, mel_spectrograms, 
                 pad_token_id=0, mel_pad_value=-100):
        self.tokenized_texts = tokenized_texts
        self.mel_spectrograms = mel_spectrograms
        self.pad_token_id = pad_token_id
        self.mel_pad_value = mel_pad_value
        
        self.n_mels = mel_spectrograms[0].shape[1]

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return (
            torch.LongTensor(self.tokenized_texts[idx]),
            torch.FloatTensor(self.mel_spectrograms[idx])
        )

def collate_fn(batch):
    texts, mels = zip(*batch)

    # Паддинг текста до максимальной длины в батче
    max_text_len = max(t.shape[0] for t in texts)
    texts_padded = [torch.cat([t, torch.full((max_text_len - t.shape[0],), 0)]) for t in texts]
    
    # Паддинг мел-спектрограмм
    max_mel_len = max(m.shape[0] for m in mels)
    mels_padded = [torch.cat([m, torch.full((max_mel_len - m.shape[0], m.shape[1]), -100)]) for m in mels]

    return torch.stack(texts_padded), torch.stack(mels_padded)

In [117]:
tokenized_texts = df['token_ids']
mel_spectrograms = df['mel_spec']


In [118]:
# Создаем DataLoader
dataset = TTSDataset(tokenized_texts, mel_spectrograms)
dataloader = DataLoader(dataset, batch_size=16, collate_fn=collate_fn)

In [165]:
dataset.__getitem__(0)[0]

tensor([2106, 1888, 6009, 1888, 5554, 1888,   89, 1888,  152, 1888, 2463, 1888,
        3958, 1888,  981, 1888, 2506, 6674, 6674, 6674, 6674, 6674, 6674, 6674,
        6674, 6674, 6674])

In [119]:
# Проверяем один пример
tokens, mel = dataset[0]
print("Пример 0:")
print(f"Токены: {tokens} (длина: {len(tokens)})")
print(f"Спектрограмма: {mel.shape}")

Пример 0:
Токены: tensor([2106, 1888, 6009, 1888, 5554, 1888,   89, 1888,  152, 1888, 2463, 1888,
        3958, 1888,  981, 1888, 2506, 6674, 6674, 6674, 6674, 6674, 6674, 6674,
        6674, 6674, 6674]) (длина: 27)
Спектрограмма: torch.Size([80, 429])


In [126]:
# Создаем DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=10,
    shuffle=True,
    collate_fn=collate_fn
)


In [127]:
# Проверяем батч
for batch_idx, (tokens_batch, mels_batch) in enumerate(dataloader):
    print(f"\nБатч {batch_idx}:")
    print(f"Токены: {tokens_batch.shape}")
    print(f"Спектрограммы: {mels_batch.shape}")
    break


Батч 0:
Токены: torch.Size([10, 27])
Спектрограммы: torch.Size([10, 80, 429])


In [128]:
tokenizer = Tokenizer()

d:\Helper\MLBazyak\chemp\Speech-synthesis\Module2


In [129]:
print(f"Размер словаря: {len(tokenizer.text_to_ids_voc)}")

Размер словаря: 6675


In [130]:
import torch.optim as optim

# Параметры
vocab_size = 6675  # размер словаря
n_mels = 80
time_steps = 429

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создание модели
model = TTSTransformer(vocab_size=vocab_size, d_model=256, n_mels=n_mels, time_steps=time_steps).to(device)

# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Функция потерь (MSE, так как предсказываем спектрограммы)
criterion = nn.MSELoss()


In [134]:
from tqdm import tqdm  # Импортируем tqdm

num_epochs = 10  # Количество эпох
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Прогресс-бар для всех батчей в эпохе
    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for batch_idx, (texts, mels) in enumerate(pbar):
            texts, mels = texts.to(device), mels.to(device)

            optimizer.zero_grad()

            # Прогон через модель
            pred_mels = model(texts)  # Выход (batch, time_steps, n_mels)
            
            # Транспонируем для корректного сопоставления размерностей
            pred_mels = pred_mels.permute(0, 2, 1)  # (batch, n_mels, time_steps)

            # Функция потерь (например, MSE)
            loss = criterion(pred_mels, mels)
            loss.backward()

            optimizer.step()
            total_loss += loss.item()

            # Обновляем описание прогресс-бара
            pbar.set_postfix(loss=loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / len(dataloader):.4f}")


Epoch 1/10: 100%|██████████| 234/234 [00:23<00:00,  9.98batch/s, loss=9.64]


Epoch [1/10], Average Loss: 10.8436


Epoch 2/10: 100%|██████████| 234/234 [00:23<00:00,  9.88batch/s, loss=8.25]


Epoch [2/10], Average Loss: 10.5748


Epoch 3/10: 100%|██████████| 234/234 [00:23<00:00, 10.06batch/s, loss=8.14]


...

Epoch [8/10], Average Loss: 11.6383


Epoch 9/10: 100%|██████████| 234/234 [00:23<00:00,  9.98batch/s, loss=8.86]


Epoch [9/10], Average Loss: 11.5752


Epoch 10/10: 100%|██████████| 234/234 [00:24<00:00,  9.69batch/s, loss=12.5]

Epoch [10/10], Average Loss: 11.9465


In [135]:
import librosa

In [136]:
def mel_to_audio(mel_spec, sr=16000, n_fft=1024, hop_length=256, n_mels=80):
    # 1. Отмена логарифмирования
    stft = librosa.feature.inverse.mel_to_stft(
        np.exp(mel_spec) - 1e-6,
        sr=sr,
        n_fft=n_fft,
        fmin=50,
        fmax=8000
    )
    
    audio = librosa.griffinlim(
        stft,
        n_iter=100,
        hop_length=hop_length,
        win_length=n_fft
    )
    
    # 3. Нормализация (чтобы избежать клиппинга)
    audio = librosa.util.normalize(audio) * 0.9
    
    return audio

In [162]:
Audio(mel_to_audio(mel_spectrograms[0]), rate=16000)

In [171]:
def token_padding(token_ids: list, vocab: dict = tokenizer.text_to_ids_voc, max_len: int = 27, ):
    while len(token_ids) != max_len:
        token_ids.append(vocab.get('<pad>'))
    return token_ids


In [174]:
text = 'Что делаешь как дела как самочувствие ля ля ля ?'
token_text = tokenizer.text_to_ids(text)
text = token_padding(token_text)
print((token_padding(token_text)))

[1505, 3734, 6673, 3734, 280, 3734, 3430, 3734, 280, 3734, 6673, 3734, 6673, 3734, 6673, 3734, 6673, 6674, 6674, 6674, 6674, 6674, 6674, 6674, 6674, 6674, 6674]


In [175]:
# Получаем предсказанные мел-спектрограммы от модели
model.eval()
with torch.no_grad():
    pred_mels = model(texts)  # Получаем предсказание из модели

In [176]:
pred_mels = np.array(pred_mels)
pred_mels

array([[[-1.8097982 , -0.5742651 , -0.60123587, ..., -4.807556  ,
         -5.169196  , -5.846808  ],
        [-1.9402641 , -0.5850991 , -0.58643764, ..., -5.1067796 ,
         -5.5155334 , -6.226628  ],
        [-2.07073   , -0.5959331 , -0.5716395 , ..., -5.4060035 ,
         -5.8618717 , -6.606449  ],
        ...,
        [-0.09417023,  0.20245843,  0.10469569, ..., -0.03337219,
         -0.23016581, -0.17344406],
        [-0.09417023,  0.20245843,  0.10469569, ..., -0.03337219,
         -0.23016581, -0.17344406],
        [-0.09417023,  0.20245843,  0.10469569, ..., -0.03337219,
         -0.23016581, -0.17344406]],

       [[-1.8066242 , -0.57346094, -0.600243  , ..., -4.7990713 ,
         -5.1600266 , -5.8361974 ],
        [-1.9372828 , -0.5843438 , -0.58550507, ..., -5.09881   ,
         -5.506921  , -6.216662  ],
        [-2.0679417 , -0.5952266 , -0.5707672 , ..., -5.3985496 ,
         -5.8538165 , -6.5971274 ],
        ...,
        [-0.09417023,  0.20245843,  0.10469569, ..., -

In [177]:
y = mel_to_audio(pred_mels)
y

d:\Helper\MLBazyak\chemp\.venv\Lib\site-packages\librosa\feature\inverse.py:99: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(


array([[-0.07025098,  0.2814779 , -0.11325296, ..., -0.13439319,
        -0.08640768,  0.46071476],
       [-0.0553559 , -0.00180003, -0.3377402 , ...,  0.45667332,
        -0.23133554, -0.9       ],
       [-0.9       ,  0.9       , -0.28483906, ...,  0.05272847,
        -0.6983859 ,  0.4167931 ],
       ...,
       [ 0.42864975, -0.10938919,  0.70933616, ..., -0.3675556 ,
         0.5066692 ,  0.53597397],
       [ 0.17579968,  0.04355725, -0.9       , ..., -0.80955625,
         0.4187681 , -0.24605756],
       [ 0.45693195, -0.68328214,  0.03542919, ...,  0.09581456,
        -0.01687393,  0.32521015]], dtype=float32)

In [156]:
import librosa
import numpy as np
import soundfile as sf
from pydub import AudioSegment



# Если это мел-спектрограмма, используем Griffen-Lim для восстановления аудио
# Для этого сначала делаем обратное преобразование мел-спектрограммы
# Мел-спектрограмма должна быть преобразована обратно в амплитудную спектрограмму

# Предположим, что у нас есть спектрограмма в мел-формате
# Сначала делаем обратное преобразование:
# Важно, чтобы у нас были такие параметры, как частота дискретизации (rate) и количество мел-банков

# Размерность спектрограммы
n_mels = y.shape[0]
n_fft = 2048  # стандартный размер БПФ
hop_length = 512  # длина шага

# Инвертируем мел-спектрограмму в аудио
audio = librosa.feature.inverse.mel_to_audio(y, sr=16000, n_fft=n_fft, hop_length=hop_length)

# Сохраняем результат в WAV
sf.write('generated_audio.wav', audio, 16000)

# Если нужно сохранить в MP3
audio_segment = AudioSegment.from_wav('generated_audio.wav')
audio_segment.export('generated_audio.mp3', format='mp3')


MemoryError: Unable to allocate 147. GiB for an array with shape (19731447825,) and data type float64

In [150]:
from IPython.display import Audio

In [178]:
# Инвертируем мел-спектрограмму в аудио
audio = librosa.feature.inverse.mel_to_audio(y, sr=16000, n_fft=n_fft, hop_length=hop_length)
audio

MemoryError: Unable to allocate 147. GiB for an array with shape (19731447825,) and data type float64

In [179]:
Audio(y, rate=16000)

In [153]:
import soundfile as sf
from pydub import AudioSegment
import numpy as np

# Проверим размерность y
if len(y.shape) == 1:
    y = np.expand_dims(y, axis=-1)  # Преобразуем в двумерный массив (samples, 1)

# Сначала сохраняем аудио в формате WAV
sf.write('generated_audio.wav', y, 16000)

# Загружаем аудио в pydub
audio_segment = AudioSegment.from_wav('generated_audio.wav')

# Сохраняем в MP3
audio_segment.export('generated_audio.mp3', format='mp3')


LibsndfileError: Error opening 'generated_audio.wav': Format not recognised.